<a href="https://colab.research.google.com/github/student64-ahmadi/Impulsive-Hamiltonian-Dynamics/blob/main/article_pysique_quantique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qutip

In [ ]:
pip install qutip numpy matplotlib

In [ ]:
pip install numpy matplotlib

In [ ]:
!pip install qutip==4.7.3

In [ ]:
%env PYTHONUTF8=1
%env PYTHONOPTIMIZE=0


env: PYTHONUTF8=1
env: PYTHONOPTIMIZE=0


In [ ]:
import os
os._exit(00)  # Force un redémarrage


In [ ]:
# -*- coding: utf-8 -*-
"""
Simulation Quantique Minimaliste - Contrôle de la Décohérence
Auteur: Ahmadi Othman
Date: 2023-10-15
"""
import os
os._exit(00)  # Force un redémarrage

import numpy as np
import matplotlib.pyplot as plt
from qutip import sigmax, sigmaz, sigmam, qeye, tensor, basis, mesolve, expect, fock_dm
import os
import zipfile

# =============================================================================
# 1. Configuration globale
# =============================================================================
CONFIG = {
    "params": {
        "J": 1.0,                   # Couplage d'échange
        "mu": 1.0,                  # Amplitude impulsion
        "gamma": 0.1,               # Taux de dissipation
        "t_event": 5.0,             # Instant de l'impulsion
        "sigma": 0.01,              # Largeur gaussienne
        "tlist": np.linspace(0, 10, 1000)  # Grille temporelle
    },
    "output": {
        "fig_dir": "figures",       # Répertoire de sortie
        "zip_file": "quantum_figures.zip"  # Archive des figures
    }
}

# =============================================================================
# 2. Définitions des fonctions principales
# =============================================================================
def setup_system():
    """Initialise les opérateurs quantiques"""
    sx, sz = sigmax(), sigmaz()
    I = qeye(2)
    return {
        "H0": -CONFIG["params"]["J"] * tensor(sx, sx),
        "H_imp": tensor(sz, I),
        "c_ops": [np.sqrt(CONFIG["params"]["gamma"]) * tensor(sigmam(), I)]
    }

def gaussian_pulse(t, args):
    """Profil temporel gaussien"""
    return (CONFIG["params"]["mu"] *
            np.exp(-(t - CONFIG["params"]["t_event"])**2 /
            (2 * CONFIG["params"]["sigma"]**2)))

def run_simulation():
    """Exécute la simulation principale"""
    system = setup_system()
    H = [system["H0"], [system["H_imp"], gaussian_pulse]]  # Correction de la syntaxe
    psi0 = tensor(basis(2, 0), basis(2, 0))
    return mesolve(H, psi0, CONFIG["params"]["tlist"], system["c_ops"], [])

# =============================================================================
# 3. Fonctions de visualisation
# =============================================================================
def plot_dynamics(result):
    """Génère les figures de dynamique quantique"""
    fig = plt.figure(figsize=(10, 12))

    # Cohérences
    ax1 = plt.subplot(311)
    coherences = [np.abs(rho.ptrace(0)[0,1]) for rho in result.states]
    ax1.plot(CONFIG["params"]["tlist"], coherences, 'b', label='Cohérence |ρ₀₁|')
    ax1.axvline(CONFIG["params"]["t_event"], color='r', linestyle='--', label='Impulsion')
    ax1.set(xlabel='Temps (ns)', ylabel='Amplitude', title='Dynamique des cohérences')
    ax1.legend()

    # Populations
    ax2 = plt.subplot(312)
    pop00 = [expect(tensor(fock_dm(2,0), fock_dm(2,0)), rho) for rho in result.states]
    pop11 = [expect(tensor(fock_dm(2,1), fock_dm(2,1)), rho) for rho in result.states]
    ax2.plot(CONFIG["params"]["tlist"], pop00, 'g', label='Population |00⟩')
    ax2.plot(CONFIG["params"]["tlist"], pop11, 'm', label='Population |11⟩')
    ax2.set(xlabel='Temps (ns)', ylabel='Population', title='Évolution des populations')
    ax2.legend()

    # Pureté
    ax3 = plt.subplot(313)
    purity = [(rho * rho).tr().real for rho in result.states]
    ax3.plot(CONFIG["params"]["tlist"], purity, 'k', label='Pureté Tr(ρ²)')
    ax3.set(xlabel='Temps (ns)', ylabel='Pureté', title='Évolution de la pureté')
    ax3.legend()

    plt.tight_layout()
    return fig

# =============================================================================
# 4. Gestion des sorties
# =============================================================================
def save_figures(fig, filename):
    """Sauvegarde les figures dans le répertoire dédié"""
    if not os.path.exists(CONFIG["output"]["fig_dir"]):
        os.makedirs(CONFIG["output"]["fig_dir"])
    fig.savefig(os.path.join(CONFIG["output"]["fig_dir"], filename),
                dpi=300, bbox_inches='tight')
    plt.close(fig)

def create_zip_archive():
    """Crée une archive ZIP des figures"""
    with zipfile.ZipFile(CONFIG["output"]["zip_file"], 'w') as zipf:
        for file in os.listdir(CONFIG["output"]["fig_dir"]):
            zipf.write(os.path.join(CONFIG["output"]["fig_dir"], file), file)
    print(f"Archive créée : {CONFIG['output']['zip_file']}")

# =============================================================================
# 5. Exécution principale
# =============================================================================
if __name__ == "__main__":
    # Simulation
    result = run_simulation()

    # Génération des figures
    dynamics_fig = plot_dynamics(result)

    # Sauvegarde
    save_figures(dynamics_fig, "quantum_dynamics.pdf")
    create_zip_archive()

    # Téléchargement
    try:
        from google.colab import files
        files.download(CONFIG["output"]["zip_file"])
    except ImportError:
        print("Exécution locale détectée : le téléchargement automatique n'est pas disponible.")


In [3]:
pip install seaborn

In [8]:
# -*- coding: utf-8 -*-
"""
Génération des figures pour l'article sur le contrôle de la décohérence
Auteur : Ahmadi Othman
Date : 15/03/2025
"""

import numpy as np
import matplotlib.pyplot as plt
from qutip import *
import os
from matplotlib.backends.backend_pdf import PdfPages

# =============================================================================
# 1. Configuration générale
# =============================================================================

# Paramètres physiques
params = {
    "J": 1.0,          # GHz - Couplage d'échange
    "mu": 1.0,         # GHz - Amplitude impulsion
    "gamma": 0.1,      # GHz - Taux de relaxation
    "t_event": 5.0,    # ns - Instant de l'impulsion
    "sigma": 0.01,     # ns - Largeur gaussienne
    "tlist": np.linspace(0, 10, 1000)  # Grille temporelle
}

# Configuration des figures
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
plt.rcParams.update({
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'lines.linewidth': 2,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.figsize': (8, 6),
    'text.usetex': False,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.edgecolor': '#333333',
    'axes.facecolor': 'white'
})

# Création des dossiers de sortie
os.makedirs('figures', exist_ok=True)

# =============================================================================
# 2. Définitions du système quantique
# =============================================================================

# Opérateurs quantiques
sx = sigmax()
sz = sigmaz()
I = qeye(2)

# Hamiltonien du système
H0 = -params["J"] * tensor(sx, sx)

# Fonction impulsion gaussienne
def gaussian_pulse(t, args):
    return (params["mu"] *
            np.exp(-(t - params["t_event"])**2 / (2 * params["sigma"]**2)))

# Hamiltonien complet
H = [H0, [tensor(sz, I), gaussian_pulse]]

# Opérateurs de collapse
c_ops = [np.sqrt(params["gamma"]) * tensor(sigmam(), I)]

# État initial
psi0 = (tensor(basis(2,0), basis(2,0)) + tensor(basis(2,1), basis(2,1))).unit()
rho0 = ket2dm(psi0)

# =============================================================================
# 3. Simulation principale
# =============================================================================

# Résolution de l'équation maîtresse
result = mesolve(H, rho0, params["tlist"], c_ops, [])

# Calcul des observables
coherences = [np.abs(state.ptrace(0)[0,1]) for state in result.states]
pop00 = [expect(tensor(fock_dm(2,0), fock_dm(2,0)), state) for state in result.states]
pop11 = [expect(tensor(fock_dm(2,1), fock_dm(2,1)), state) for state in result.states]
purity = [(state*state).tr().real for state in result.states]

# =============================================================================
# 4. Génération des figures principales
# =============================================================================

with PdfPages('figures/all_figures.pdf') as pdf:

    # Figure 1 : Dynamique des cohérences
    fig1 = plt.figure(figsize=(10, 6))
    plt.plot(params["tlist"], coherences, color=colors[0], label='|ρ₀₁|')
    plt.axvline(params["t_event"], color='k', linestyle='--', label='Impulsion')
    plt.xlabel('Temps (ns)')
    plt.ylabel('Cohérence')
    plt.title('Dynamique des cohérences')
    plt.legend(frameon=True, facecolor='white', edgecolor='black')
    pdf.savefig(fig1)
    plt.close()

    # Figure 2 : Transfert de population
    fig2 = plt.figure(figsize=(10, 6))
    plt.plot(params["tlist"], pop00, color=colors[1], label='Population |00⟩')
    plt.plot(params["tlist"], pop11, color=colors[2], label='Population |11⟩')
    plt.xlabel('Temps (ns)')
    plt.ylabel('Population')
    plt.title('Transfert de population')
    plt.legend(frameon=True, facecolor='white', edgecolor='black')
    pdf.savefig(fig2)
    plt.close()

    # Figure 3 : Évolution de la pureté
    fig3 = plt.figure(figsize=(10, 6))
    plt.plot(params["tlist"], purity, color=colors[3], label='Tr(ρ²)')
    plt.axvline(params["t_event"], color='k', linestyle='--', label='Impulsion')
    plt.xlabel('Temps (ns)')
    plt.ylabel('Pureté')
    plt.title('Évolution de la pureté quantique')
    plt.legend(frameon=True, facecolor='white', edgecolor='black')
    pdf.savefig(fig3)
    plt.close()

    # Figure 4 : Sensibilité à l'amplitude μ
    fig4 = plt.figure(figsize=(10, 6))
    mu_values = [0.5, 1.0, 1.5]
    for mu in mu_values:
        params["mu"] = mu
        result_mu = mesolve(H, rho0, params["tlist"], c_ops, [])
        purity_mu = [(state*state).tr().real for state in result_mu.states]
        plt.plot(params["tlist"], purity_mu, label=f'μ = {mu} GHz')
    plt.xlabel('Temps (ns)')
    plt.ylabel('Pureté')
    plt.title('Sensibilité à l\'amplitude μ')
    plt.legend(frameon=True, facecolor='white', edgecolor='black')
    pdf.savefig(fig4)
    plt.close()

    # =============================================================================
    # 5. Figures expérimentales supplémentaires
    # =============================================================================

    # Données simulées pour l'analyse spectrale
    time_exp = np.linspace(10, 20, 100)
    population_exp = 0.4 + 0.1 * np.sin(2 * np.pi * 0.35 * time_exp)  # Oscillations à f = 0.35 Hz
    frequency = np.linspace(0.3, 0.5, 100)
    amplitude_spectrum = np.exp(-((frequency - 0.35)**2) / (2 * 0.02**2))  # Pic spectral

    # Figure 5 : Dynamique de population expérimentale
    fig5 = plt.figure(figsize=(10, 6))
    plt.plot(time_exp, population_exp, color=colors[0], label='Données expérimentales')
    plt.axvline(15, color='k', linestyle='--', label='Impulsion')
    plt.xlabel('Temps (ns)')
    plt.ylabel('Population normalisée')
    plt.title('Dynamique de population sous impulsion')
    plt.legend(frameon=True, facecolor='white', edgecolor='black')
    pdf.savefig(fig5)
    plt.close()

    # Figure 6 : Analyse spectrale des oscillations
    fig6 = plt.figure(figsize=(10, 6))
    plt.plot(frequency, amplitude_spectrum, color=colors[1], label='Spectre mesuré')
    plt.axvline(0.35, color='k', linestyle='--', label='f = 0.35 ± 0.02 Hz')
    plt.xlabel('Fréquence (Hz)')
    plt.ylabel('Amplitude normalisée')
    plt.title('Analyse spectrale des oscillations')
    plt.legend(frameon=True, facecolor='white', edgecolor='black')
    pdf.savefig(fig6)
    plt.close()

print("Génération terminée ! Téléchargez le dossier 'figures' depuis Colab.")

/usr/local/lib/python3.11/dist-packages/qutip/solver/solver_base.py:576: FutureWarning: e_ops will be keyword only from qutip 5.3 for all solver
  warnings.warn(


Génération terminée ! Téléchargez le dossier 'figures' depuis Colab.
